Загружаем исходные данные в colab.

In [ ]:
from google.colab import files
uploaded = files.upload()

Загружаем данные в датафрейм

In [ ]:
import io
import pandas as pd
import numpy as np
import plotly.express as px

df = pd.read_csv(io.BytesIO(uploaded['dataset.csv']))

df

,event_date,app_version,country_code,install_date,media_source,campaign_name,appsflyer_id,is_organic,payments,revenue_usd,ad_revenue_usd,sessions,playtime
0,2022-04-09,0.1.75.1,SG,2022-04-09,bytedanceglobal_int,EM_AND_SEA_Purchase_IT2_0604,1649548002569-6720420720726321605,False,0,0.0,0.000000,1,4
1,2022-04-09,0.1.75.1,US,2022-04-09,bytedanceglobal_int,EM_AND_US_Purchase_IT1_2703,1649545238733-4551347935758619924,False,0,0.0,0.000000,1,2
2,2022-04-09,0.1.75.1,US,2022-04-09,bytedanceglobal_int,EM_AND_US_Purchase_IT1_2703,1649540322226-5366969563919899948,False,0,0.0,0.000000,1,2
3,2022-04-09,0.1.75.1,UK,2022-04-07,bytedanceglobal_int,EM_AND_T1_Purchase_IT2_0604,1649337475698-617904100123526593,False,0,0.0,0.000000,1,1
4,2022-04-09,0.1.75.1,US,2022-04-09,bytedanceglobal_int,EM_AND_US_Purchase_IT1_2703,1649544985557-9160306254813664580,False,0,0.0,0.000000,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
690370,2022-07-31,NaN,PH,2022-07-28,organic,organic,1659003289354-9130797695840500796,True,0,0.0,0.002614,1,1
690371,2022-07-31,NaN,AE,2022-07-28,organic,organic,1659020308480-7169513513931602186,True,0,0.0,0.012164,1,1
690372,2022-07-31,NaN,DZ,2022-07-29,organic,organic,1659059933950-196412188438371439,True,0,0.0,0.000195,1,1
690373,2022-07-31,NaN,FR,2022-07-29,organic,organic,1659086493021-6361713490086303799,True,0,0.0,0.021129,1,1


In [ ]:
df['event_date'] = pd.to_datetime(df['event_date'])
df['install_date'] = pd.to_datetime(df['install_date'])
df.dtypes

event_date        datetime64[ns]
app_version               object
country_code              object
install_date      datetime64[ns]
media_source              object
campaign_name             object
appsflyer_id              object
is_organic                  bool
payments                   int64
revenue_usd              float64
ad_revenue_usd           float64
sessions                   int64
playtime                   int64
dtype: object

Сегодня вспомним основные монетизационные метрики - ARPU, ARPPU, ARPDAU, average check.

Посмотрим средний ARPU по всем данным.

In [ ]:
ARPU = df['revenue_usd'].sum() / df['appsflyer_id'].nunique()
ARPU

0.12910787623317102

Однако, это не полный ARPU проекта - необходимо учесть также деньги с рекламной монетизации, добавим их в расчет и посмотрим насколько изменится значение ARPU.

In [ ]:
ARPU_full = (df['revenue_usd'].sum() + df['ad_revenue_usd'].sum()) / df['appsflyer_id'].nunique()
ARPU_full

0.15106321905544917

Рассчитаем средний ARPPU - для его расчета уже используем только выручку с внутреигровых покупок.

Для начала необходимо рассчитать число платящих.

In [ ]:
payers = df[df['payments']>0]['appsflyer_id'].nunique()
ARPPU = df['revenue_usd'].sum() / payers
ARPPU

7.8739909192224

Посмотрим какой гео дает самый высокий ARPPU

In [ ]:
df_arppu = df[df['payments']>0].groupby('country_code', as_index = False).agg({'revenue_usd':'sum', 'appsflyer_id':'nunique'}).rename(columns = {'appsflyer_id':'payers'})
df_arppu['arppu'] = df_arppu['revenue_usd'] / df_arppu['payers']

df_arppu.sort_values(by = 'arppu', ascending = False).head(10)

,country_code,revenue_usd,payers,arppu
66,MV,74.610000,2,37.305000
84,RU,465.363881,13,35.797222
63,MP,28.950000,1,28.950000
31,GN,19.128512,1,19.128512
28,FO,37.054787,2,18.527393
74,PE,1138.952969,66,17.256863
2,AR,17.110000,1,17.110000
44,IS,34.056388,2,17.028194
13,CH,360.995362,27,13.370199
35,GY,12.970000,1,12.970000


Попробуем отсортировать этот же датафрейм по выручке

In [ ]:
df_arppu.sort_values(by = 'revenue_usd', ascending = False).head(10)

,country_code,revenue_usd,payers,arppu
98,US,17661.187734,1952,9.047740
97,UK,2764.555323,326,8.480231
21,DE,2196.258972,257,8.545755
29,FR,1850.688861,204,9.072004
12,CA,1464.428101,156,9.387360
39,ID,1463.859155,297,4.928819
68,MY,1450.416814,328,4.422002
74,PE,1138.952969,66,17.256863
4,AU,1124.813919,103,10.920523
75,PH,893.948975,264,3.386170


Сравните полученные результаты, какие выводы можно сделать?

Одним из наиболее частых вариантов использования ARPU является расчет ARPDAU - причем зачастую его рассчитывают отдельно по выручке с ин-аппов и рекламы.

In [ ]:
df_arpdau = df.groupby('event_date').agg({'appsflyer_id':'nunique', 'revenue_usd':'sum', 'ad_revenue_usd':'sum'}).rename(columns = {'appsflyer_id':'dau'})
df_arpdau['iap_arpdau'] = df_arpdau['revenue_usd'] / df_arpdau['dau']
df_arpdau['ad_arpdau'] = df_arpdau['ad_revenue_usd'] / df_arpdau['dau']

df_arpdau

,dau,revenue_usd,ad_revenue_usd,iap_arpdau,ad_arpdau
event_date,,,,,
2022-04-01,1452,173.264061,27.533387,0.119328,0.018962
2022-04-02,1526,115.421469,28.379664,0.075637,0.018597
2022-04-03,1419,114.398078,29.971379,0.080619,0.021121
2022-04-04,768,121.715021,22.334192,0.158483,0.029081
2022-04-05,623,31.138029,18.404850,0.049981,0.029542
...,...,...,...,...,...
2022-07-27,8207,326.847662,58.154378,0.039825,0.007086
2022-07-28,8263,317.396381,50.532636,0.038412,0.006116
2022-07-29,8970,290.280088,50.500314,0.032361,0.005630


In [ ]:
fig_iap_arpdau = px.line(df_arpdau, x = df_arpdau.index, y = 'iap_arpdau')
fig_iap_arpdau.show()

In [ ]:
fig_ad_arpdau = px.line(df_arpdau, x = df_arpdau.index, y = 'ad_arpdau')
fig_ad_arpdau.show()

Какие выводы можно сделать по полученным графикам? Анализ каких метрик вы предложили бы провести, чтобы найти причины наблюдаемых изменений?

Попробуем посчитать монетизационные метрики в разрезе каналов привлечения пользователей.

In [ ]:
df_channel = df.groupby('media_source', as_index=False).agg({'appsflyer_id':'nunique', 'revenue_usd':'sum', 'ad_revenue_usd':'sum', 'payments':'sum'}).rename(columns = {'appsflyer_id':'users'})
df_channel

,media_source,users,revenue_usd,ad_revenue_usd,payments
0,Facebook Ads,2,0.000000,0.000000,0
1,bytedanceglobal_int,21682,3781.626236,971.360863,923
2,googleadwords_int,40,20.691668,3.526398,3
3,organic,310275,39047.940679,6311.981245,9914


По полученным результатам видим, что значимая часть пользователей пришли только по двум каналм - органика и тик-ток. Рассчитаем по ним монетизационные метрики.

In [ ]:
df_channel = df_channel[df_channel['media_source'].isin(['bytedanceglobal_int', 'organic'])]
df_channel['arpu'] = (df_channel['revenue_usd']+df_channel['ad_revenue_usd']) / df_channel['users']
df_channel['average_check'] = df_channel['revenue_usd'] / df_channel['users']

df_channel

,media_source,users,revenue_usd,ad_revenue_usd,payments,arpu,average_check
1,bytedanceglobal_int,21682,3781.626236,971.360863,923,0.219213,0.174413
3,organic,310275,39047.940679,6311.981245,9914,0.146193,0.125849


К полученным данным остается добавить одно значение - ARPPU, для этого нам необходимо рассчитать число платящих в каждой когорте.

In [ ]:
df_payers = df[(df['media_source'].isin(['bytedanceglobal_int', 'organic'])) & (df['payments']>0)].groupby('media_source', as_index=False).agg({'appsflyer_id':'nunique'}).rename(columns = {'appsflyer_id':'payers'})

df_channel = pd.merge(df_channel, df_payers, on='media_source')
df_channel['arppu'] = df_channel['revenue_usd'] / df_channel['payers']

df_channel

,media_source,users,revenue_usd,ad_revenue_usd,payments,arpu,average_check,payers,arppu
0,bytedanceglobal_int,21682,3781.626236,971.360863,923,0.219213,0.174413,469,8.063169
1,organic,310275,39047.940679,6311.981245,9914,0.146193,0.125849,4973,7.851989


Какие выводы можно сделать по полученным результатам?

# Домашнее задание

Попробуйте объяснить чем вызваны изменения ARPDAU в проекте.
Проанализируйте особенности монетизационных метрик по гео.

Конверсия проходит в бльшенстве случев у игроков которрые находятся 1,5-2 месяца в игре.

Со временем количество неплатящих игроков увеличилось, что привело к снижению дохода. Это может быть вязано с регионом в котором снижено кличество плательщиков. Для получения большей информации о снижении количество конверсии необходимо знать, об изменениях внутри игры.

Необходимо проерять оставерность платежей, смотря на суммы и страны из которых они поступают.

Наибольшее количество игроков находиться в Америке и Европе.